In [2]:
import os
import torch
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO

In [3]:
# Set device assuming a GPU is available
device = torch.device('cuda')

# Load pre-trained Faster R-CNN model and move it to the GPU
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
model.eval()  # Set model to evaluation mode

# Define the image transformation
transform = transforms.Compose([
    transforms.ToTensor()
])

annFile = 'coco/instances_val2017.json'
coco = COCO(annFile)

# Get all category IDs and then names
cat_ids = coco.getCatIds()
cats = coco.loadCats(cat_ids)
coco_instance_category_names = [cat['name'] for cat in cats]

print(coco_instance_category_names)

# Folder containing the uploaded images
folder_path = 'sample_images'

# Folder to save processed images. Create the folder if it doesn't exist.
processed_folder = 'processed_images'
os.makedirs(processed_folder, exist_ok=True)

# List all supported image files in the folder
supported_formats = ('.jpg', '.jpeg', '.png')
image_files = [fname for fname in os.listdir(folder_path) if fname.lower().endswith(supported_formats)]

# Loop through each image file in the folder
for fname in image_files:
    image_path = os.path.join(folder_path, fname)
    image = Image.open(image_path).convert("RGB")
    img_tensor = transform(image).to(device)

    # Run inference
    with torch.no_grad():
        predictions = model([img_tensor])

    # Create the plot
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)

    boxes = predictions[0]['boxes']
    scores = predictions[0]['scores']
    labels = predictions[0]['labels']

    # Set a confidence threshold for displaying detections
    threshold = 0.5

    for i, score in enumerate(scores):
        if score > threshold:
            box = boxes[i].cpu().numpy()
            label_idx = labels[i].cpu().numpy()
            label_name = coco_instance_category_names[label_idx]
            x1, y1, x2, y2 = box
            # Draw bounding box
            rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect)
            # Annotate with label and score
            text = f'{label_name}: {score:.2f}'
            ax.text(x1, y1, text, fontsize=12, color='yellow', backgroundcolor='black')

    ax.set_title(f'Detections for {fname}')
    plt.axis('off')

    # Save the annotated image to the processed_images folder
    save_path = os.path.join(processed_folder, f'processed_{fname}')
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)  # Close the figure to free memory

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 155MB/s]


loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: 'coco/instances_val2017.json'